# Input Generation using Basal Ganglia Data

If you have done (input tuning)[../parallel/KTH_PDC/input_tuning] and saved the input settings to the neuron's meta.json files in BasalGangliaData then you can use that to set the appropriate level of your inputs.

If you need to tune the cortical and thalamic input for your own neurons, then see [this folder](../parallel/KTH_PDC/input_tuning/) which has examples and more information in the [README.md](../parallel/KTH_PDC/input_tuning/README.md) file.

In [ ]:
import os
import numpy as np

network_path = os.path.join("networks","simple_example")
snudda_data = os.path.join("..", "..", "..", "BasalGangliaData", "data") 

## Generate the network

Here we create a network with 100 dSPN and 100 iSPN neurons. We also define two population units with 20 neurons each, by defining two spherical regions with radius 200 micrometers, and picking 20 neurons within those spheres).

The population units will be named 1 and 2.

In [ ]:
from snudda import SnuddaInit

si = SnuddaInit(network_path=network_path, random_seed=12345, snudda_data=snudda_data)
si.define_striatum(num_dSPN=100, num_iSPN=100, num_FS=0, num_LTS=0, num_ChIN=0, neuron_density=80500,
                    volume_type="cube")

# Here we add two population units
# The centre of the cube is [0.00475, 0.004, 0.00775]. num_neurons is optional
si.add_population_unit_density(structure_name="Striatum", neuron_types=["dSPN", "iSPN"], 
                               unit_centre=np.array([0.00475, 0.004, 0.00775]) -np.array([0,50e-6,150e-6]),
                               probability_function="(d < 200e-6)*1", num_neurons=20)
si.add_population_unit_density(structure_name="Striatum", neuron_types=["dSPN", "iSPN"], 
                               unit_centre=np.array([0.00475, 0.004, 0.00775]) -np.array([150e-6,0,0]),
                               probability_function="(d < 200e-6) * 1", num_neurons=20)
si.write_json()

In [ ]:
from snudda import Snudda
snd = Snudda(network_path=network_path)
snd.create_network()

## Define input

The input tuning has set up four inputs for each neuron ```cortical_background```, ```thalamic_background```, ```cortical``` and ```thalamic```. The first two are background input at a low frequency (2Hz) which should bring the neurons to just below firing threshold. The last two inputs are by default set to 0Hz input frequency, but they are tuned so that when the input frequency is 10Hz the neurons should spike approximately at 40Hz. There are some variation depending on spike patterns and synapse location.

So what we want to do is modify the input frequency of the ```cortical``` input so that we get the spiking we want.

In [ ]:
# Code to dispaly example JSON file
import json
with open("input_config/basal-ganglia-data-input.json", "r") as f:
    params = json.load(f)
    print(json.dumps(params, indent=4))


Here we can see that for the ```dSPN``` neurons we define two inputs: ```cortical:1``` and ```cortical:2``` that will be based on the ```cortical``` input. Everything after the ```:``` is ignored when finding a ```meta.json``` input to match with. The reason we don't simply call them both ```cortical``` is that each name has to be unique.

If we look in one of the ```meta.json``` files we can see (exerpt from the file), which will act as a template for our ```cortical:1``` input to the neurons:

```                "cortical": {
                    "generator": "poisson",
                    "type": "AMPA_NMDA",
                    "synapse_density": "1.15*0.05/(1+exp(-(d-30e-6)/5e-6))",
                    "num_inputs": 100,
                    "frequency": 0.0,
                    "population_unit_correlation": 0.0,
                    "jitter": 0.0,
                    "conductance": 5e-10,
                    "mod_file": "tmGlut",
                    "parameter_file": "$DATA/synapses/striatum/M1RH_Analysis_190925.h5-parameters-MS.json"
                },
```

For a ```dSPN``` belonging to population unit 1 we will change the frequency to 5Hz. We will also add ```start``` and ```end``` times to the input.

In [ ]:
import os
from snudda.input import SnuddaInput
si = SnuddaInput(network_path=network_path, 
                 input_config_file=os.path.join("input_config","basal-ganglia-data-input.json"),
                 verbose=False)
si.generate()

## Plotting the input

Let's look at the input for population unit 0 (ie. neurons not allocated to any population unit), and the two populations 1 and 2. Here we plot two neurons from each group.

In [ ]:
from snudda.plotting import PlotInput
input_file = os.path.join(network_path, "input-spikes.hdf5")
spi = PlotInput(input_file)

### No population unit (id 0)

Here you can see that the two neurons (44) and (168) have both ```cortical_background``` and ```thalamic_background``` input, and that neuron 44 has many more synapses than neuron 168.

In [ ]:
spi.plot_input_population_unit(population_unit_id=0, num_neurons=2, neuron_type="dSPN", fig_size=(15,5))

### Population unit 1

Here we see that the ```cortical``` inputs are active between 1 and 3, and 4 and 6 seconds. While the ```cortical_background``` and ```thalamic background``` are active throught the simulation.

In [ ]:
spi.plot_input_population_unit(population_unit_id=1, num_neurons=2, neuron_type="dSPN", fig_size=(15,5))

### Population unit 2

Here we see that the ```cortical``` inputs are active between 4 and 6 seconds. Again the ```cortical_background``` and ```thalamic background``` are active throught the simulation.

In [ ]:
spi.plot_input_population_unit(population_unit_id=2, num_neurons=2, neuron_type="dSPN", fig_size=(15,5))

## Run the simulation:

In [ ]:
n_cores = 8
duration = 10
os.system(f"mpiexec -n {n_cores} snudda simulate {network_path} --time {duration}")